## Replicating / trying out Kozlowski data

In [11]:
import itertools
import logging
import os

import gensim
import smart_open
import numpy as np

In [16]:
class MySentences(object):
    def __init__(self, dirname, start_year, end_year, limit=None):
        self.dirname = dirname
        self.start_year = start_year
        self.end_year = end_year
        self.limit = limit

    def __iter__(self):
        # iterate through each the compressed file directory
        for fname in os.listdir(self.dirname):
            # for each compressed file open it
            with gensim.utils.smart_open(os.path.join(self.dirname, fname)) as fin:
                for line in itertools.islice(fin, self.limit):
                    line = gensim.utils.to_unicode(line).split("\t")
                    if len(line)<3:
                        continue
                    ngram = line[0]
                    try:
                        year = int(line[1])
                    except ValueError:
                        continue
                    match_count = int(line[2])
                    if year < self.start_year or year > self.end_year:
                        continue
                    # lower case the ngram, remove pos
                    processed_ngram = [word.split("_")[0] for word in ngram.lower().split()]
                    for x in range(match_count):
                        yield processed_ngram

class MySentsSimple:
    def __init__(self, fpath, limit=None):
        self.fpath = fpath
        self.limit = limit

    def __iter__(self):
        # for each compressed file open it
        with smart_open.open(self.fpath, encoding="utf-8") as infile:
            for line in itertools.islice(infile, self.limit):
                    line = gensim.utils.to_unicode(line).split("\t")
                    if len(line)<3:
                        continue
                    ngram = line[0]
                    try:
                        year = int(line[1])
                    except ValueError:
                        continue
                    match_count = int(line[2])
                    if year < self.start_year or year > self.end_year:
                        continue
                    # lower case the ngram, remove pos
                    processed_ngram = [word.split("_")[0] for word in ngram.lower().split()]
                    for x in range(match_count):
                        yield processed_ngram

In [21]:
assert gensim.models.word2vec.FAST_VERSION > -1
##Reads ngrams one-by-one into word2vec##
zip_fpath = "../data/google_news_embedding.zip"
sentences = MySentsSimple(zip_fpath, limit=20) # a memory-friendly iterator

In [33]:
file_enc = "latin-1"
with smart_open.open(zip_fpath, 'rb', encoding=file_enc) as infile:
    print(gensim.utils.to_unicode(infile.read(1000)))
    #for line in infile:
    #    print(line)
    #    line = gensim.utils.to_unicode(line).split("\t")

PK-   	 ^mOG-Fÿÿÿÿÿÿÿÿ  GoogleNews_Embedding.csv  üñ7C   vN6E    tÿK,Í$ìíÏ¯(9Ã%ky­*
EÈ¿5okÌF 
 è¯¥ÿ=Ã=ÂÜMTõâê¬¬¿©©ªýþýüýÿ¾ÿÜÆîã?ñçøÏÇøÏçøÏ×øÏeÿ½ÌÿÎ_ºÌßºÌ_»Ìß»Ì_¼Ìß¼Ì_½Ìß½Îß½Îß½Îß½Îß½Îß½Îß½Îß½Îß½Îß½Îß½Íß½Íß½vþîmþîmþîmþîmþîmþîmþî}þî}þî}þîýø¦ówïówïówïówïówïówówówówówÇe¿û¿û¿û¿û¿û¿û¿û¿û¿û¿û<®ñüÝçüÝçüÝçüÝù»ów?æï~Ìßý¿û1÷ã¸Aów?æï~Ìßý¿û9÷sþîçüÝÏù»ów?çï~wwþîçüÝ¯ù»_ów¿æï~Íßý¿û5÷kþî×üÝ¯chcãÛ1:¶cxlÇøØ²#d;Èví$Ûñ.5Äw9Ù9ÊÎav³s #íjÇX»ír=Gêñ.Çx»îr¸Ë1ä.Ç»îrºË1ì.Ç¸»ÜÎ¼Ë1ô.ÇØ»ïr¾Ë1ü.Çø»ðrÀË1/÷ó¹9Þåc^qx9âåc(^±x9ãåÇùøïrÈË1"/Ç¼còrÊË1*/Ç°¼ãòrÌËó|w9Ææåct^áy9Æçå c^!z9Æèåãw9éå§c ^z9êå«c°^Ñz9ëåóSw9Fìå²cÌ^A{9Fíå¶cÜ^{9FîåëÎ¹é±{=Æîõ»×cì^±{=Æîõ»×cì^±{½SÜñ.ÇØ½c÷zÝë1v¯ÇØ½c÷zÝë9OeÍÇ»så9Y³å9]óå9ac÷zÝë1v¯·sÂ=Þå»×cì^±{=Æîõ»×cì^±{=Æîõ»×û9oïrÝë1v¯ÇØ½c÷zÝë1v¯ÇØ½c÷zÝëãþw9Æîõ»×cì^±{=Æîõ»×cì^±{=Æîõy®"Ç»c÷zÝë1v¯ÇØ½c÷zÝë1v¯ÇØ½c÷úq.FÇ»c÷zÝë1v¯ÇØ½


In [18]:
###Set parameters. Details here: https://radimrehurek.com/gensim/models/word2vec.html###
model = gensim.models.word2vec.Word2Vec(sentences,sg=1, vector_size=300, window=5,
                                        min_count=10, workers=10, hs=0, negative=8)
model.save('w2vmodel_ng5_'+str(year_1)+'_'+str(year_2)+'_full')
syn0_object=model.wv.syn0


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x97 in position 10: invalid start byte

In [ ]:

##output vector space##
numpy.savetxt('syn0_ngf_'+str(year_1)+'_'+str(year_2)+'_full.txt',syn0_object,delimiter=" ")

#output vocab list#
vocab_list = model.wv.index2word
for i in range(0,len(vocab_list)):
    if vocab_list[i] == '':
        vocab_list[i] = "thisisanemptytoken"+str(i)

with open('vocab_list_ngf_'+str(year_1)+'_'+str(year_2)+'_full.txt','wb') as outfile:
    for i in range(0,len(vocab_list)):
        outfile.write(vocab_list[i].encode('utf8')+"\n".encode('ascii'))